In [1]:
import findspark

In [2]:
findspark.init('/home/dhruva_stark/spark-3.4.1-bin-hadoop3')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark=SparkSession.builder.appName('nlp').getOrCreate()

23/07/12 17:04:52 WARN Utils: Your hostname, linux resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/07/12 17:04:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/12 17:04:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/12 17:04:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
data=spark.read.csv('smsspamcollection/SMSSpamCollection',inferSchema=True,sep='\t')

In [9]:
data=data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [10]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [11]:
from pyspark.sql.functions import length

In [13]:
data=data.withColumn('length',length(data['text']))

In [14]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [15]:
data.groupBy('class').mean().show()


+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [16]:
from pyspark.ml.feature import (Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer)

In [30]:
tokenizer=Tokenizer(inputCol='text',outputCol='token_text')
stop_remove=StopWordsRemover(inputCol='token_text',outputCol='stop_token')
count_vec=CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf=IDF(inputCol='c_vec',outputCol='tf_idf')
ham_spam_to_numeric=StringIndexer(inputCol='class',outputCol='label')

In [31]:
from pyspark.ml.feature import VectorAssembler

In [32]:
clean_up=VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [33]:
from pyspark.ml.classification import NaiveBayes

In [34]:
nb=NaiveBayes()

In [35]:
from pyspark.ml import Pipeline

In [36]:
data_prep_pipe=Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove,count_vec,idf,clean_up])

In [37]:
cleaner=data_prep_pipe.fit(data)

In [38]:
clean_data=cleaner.transform(data)

In [39]:
clean_data=clean_data.select('label','features')

In [40]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [41]:
training,test=clean_data.randomSplit([0.7,0.3])


In [42]:
spam_detection=nb.fit(training)

23/07/12 18:24:25 WARN DAGScheduler: Broadcasting large task binary with size 1165.0 KiB
23/07/12 18:24:26 WARN DAGScheduler: Broadcasting large task binary with size 1144.3 KiB


In [43]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [44]:
test_results=spam_detection.transform(test)

In [45]:
test_results.show()

23/07/12 18:25:51 WARN DAGScheduler: Broadcasting large task binary with size 1370.4 KiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-792.09158785104...|[1.0,2.2634040925...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-543.52906344113...|[1.0,1.0585652491...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-885.67344309047...|[1.0,2.1165772698...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1359.8683368048...|[1.0,9.5484695865...|       0.0|
|  0.0|(13424,[0,1,14,79...|[-690.13420360957...|[1.0,1.9782580870...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-668.35180908857...|[1.0,9.5598072446...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-750.20622099352...|[1.0,6.8616611811...|       0.0|
|  0.0|(13424,[0,1,27,35...|[-1473.6015225785...|[0.99999999999002...|       0.0|
|  0.0|(13424,[0,1,30,11...|[-591.97063072263...|[1.0,1.6008419471...|       0.0|
|  0.0|(13424,[0

23/07/12 18:25:52 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [47]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [55]:
acc_eval=MulticlassClassificationEvaluator()

In [56]:
acc = acc_eval.evaluate(test_results)



23/07/12 18:31:11 WARN DAGScheduler: Broadcasting large task binary with size 1374.9 KiB


In [57]:
print('ACC of NB Model')
print(acc)


ACC of NB Model
0.9314137218910742
